# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-7B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-7B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [1]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W0901 14:43:25.613000 17140 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [2]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-7b_lora_model"
# model_name = "qwen-7b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-7B" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

D:\ProgramData\miniconda3\envs\deepseek\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.8.10: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 2080 Ti. Num GPUs = 1. Max memory: 11.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu129. CUDA: 7.5. CUDA Toolkit: 12.9. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [3]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [4]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [5]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [6]:
print(response[0].split("### Response:")[1])


<think>
好，我现在需要帮助一位28岁的程序员处理他的症状。他最近工作到半夜，感觉头晕、脖子疼，有时候还恶心。首先，我要考虑他的工作情况，长时间夜班可能导致疲劳，特别是睡眠不足，这可能影响他的神经系统。

接下来，头晕和恶心可能是由于低血压引起的，尤其是在站立或转动头部时，血压骤降会引发这些症状。我应该考虑他是否有高血压史，或者其他可能导致低血压的疾病，比如肾功能不全或者甲状腺问题。

另外，脖子疼痛可能与神经受压有关，比如颈椎病或者甲状腺功能异常导致的甲状腺前叶肥大压迫神经。我需要询问他的家族史，是否有类似症状，以及是否有过颈部外伤。

还需要考虑他的饮食和生活习惯，比如咖啡因摄入是否过多，或者是否存在其他可能导致低血压的食物。此外，他是否有服用药物，特别是抗抑郁药或者抗高血压药，这些药物可能会引起低血压反应。

最后，我应该建议他进行进一步的检查，比如血压监测、血糖检测、甲状腺功能检查和颈椎评估，以确定具体原因，并制定适当的治疗方案。
</think>

根据您的描述，我需要帮助分析可能的健康状况并提供建议。以下是逐步的思考过程：

1. **症状分析**：
   - **头晕**：常见于低血压、贫血、神经压迫或脱水。
   - **恶心**：可能与低血压、饮食不当或神经系统问题有关。
   - **脖子疼痛**：可能是颈椎病、甲状腺压迫或颈部损伤。

2. **潜在原因**：
   - **低血压**：尤其在站立或头部转动时加重，考虑有无高血压病史或肾脏疾病。
   - **甲状腺功能异常**：导致甲状腺前叶肥大压迫神经，引起头晕和脖子疼痛。
   - **颈椎问题**：如颈椎病压迫神经，导致头晕和疼痛。
   - **脱水**：可能导致头晕和恶心，但通常伴有口渴和尿少。

3. **检查建议**：
   - **血压监测**：确认低血压情况。
   - **血糖检测**：排除贫血或其他代谢问题。
   - **甲状腺功能检查**：评估甲状腺状态。
   - **颈椎评估**：通过X光或MRI检查。

4. **建议措施**：
   - **休息调整**：保证充足睡眠，避免熬夜。
   - **饮食调整**：减少咖啡因摄入，增加高蛋白、高维生素食物。
   - **药物监测**：避免影响血压的药物使用。
   - **就医安排**：安排进一步检查，明确

---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [7]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [8]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [9]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [10]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [11]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-3): 4 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [12]:
from trl import SFTConfig, SFTTrainer
import sys
import os

# 手动添加 Unsloth 编译缓存目录到 Python 路径
# 注意：路径需替换为你实际的 unsloth_compiled_cache 路径（从错误日志中获取）
cache_dir = "D:\\workspace\\deepseek-quickstart\\distill\\unsloth_compiled_cache"
if cache_dir not in sys.path:
    sys.path.append(cache_dir)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [13]:
# 1. 先导入必要库，再设置环境变量（必须放在代码最开头！）
import os
os.environ["TORCHINDUCTOR_CACHE_DIR"] = "D:\\torch_temp"  # 自定义临时目录（用绝对路径，双反斜杠）
os.environ["TRITON_CACHE_DIR"] = "D:\\torch_temp\\triton"  # Triton 单独缓存目录

# 2. 确保目录存在（不存在则创建）
if not os.path.exists(os.environ["TORCHINDUCTOR_CACHE_DIR"]):
    os.makedirs(os.environ["TORCHINDUCTOR_CACHE_DIR"], exist_ok=True)
if not os.path.exists(os.environ["TRITON_CACHE_DIR"]):
    os.makedirs(os.environ["TRITON_CACHE_DIR"], exist_ok=True)

os.environ["CUDA_LAUNCH_BLOCKING"] = "0"  # 关闭同步阻塞（默认应该是0）
os.environ["TORCH_CUDNN_BENCHMARK"] = "1"  # 启用 cudnn 自动调优
os.environ["NCCL_DEBUG"] = "WARN"  # 仅输出 NCCL 警告，减少日志开销

torch.cuda.empty_cache()  # 清空未使用的GPU缓存

print("模型设备：", model.device)  # 应显示cuda:0

trainer.accelerator.state.use_cuda_graph = True



trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

模型设备： cuda:0


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,entropy
1,3.185200,0
2,3.038100,No Log
3,3.215300,No Log
4,3.127600,No Log
5,3.009500,No Log
6,2.909300,No Log
7,2.836400,No Log
8,2.713600,No Log
9,2.697500,No Log
10,2.444100,No Log


TrainOutput(global_step=60, training_loss=2.181680061419805, metrics={'train_runtime': 825.1748, 'train_samples_per_second': 0.582, 'train_steps_per_second': 0.073, 'total_flos': 1.6973533272342528e+16, 'train_loss': 2.181680061419805, 'epoch': 0.02379535990481856})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [14]:
model.save_pretrained("qwen-7b_lora_model")

In [15]:
tokenizer.save_pretrained("qwen-7b_lora_model")

('qwen-7b_lora_model\\tokenizer_config.json',
 'qwen-7b_lora_model\\special_tokens_map.json',
 'qwen-7b_lora_model\\chat_template.jinja',
 'qwen-7b_lora_model\\tokenizer.json')

In [16]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [18]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这个患者已经发病了五天，腹痛稍微减轻了，但还是有些发热，看来情况不是特别严重。现在右下腹有一个包块，感觉有点大，而且压痛还比较明显。我得想想，这种情况下应该怎么做。

首先，我得考虑包块的性质。包块可能有几种情况，比如是脂肪瘤、良性的肿瘤或者恶性肿瘤。如果是良性的，比如脂肪瘤，一般不会有疼痛，但这个患者有压痛，说明可能不是脂肪瘤。那如果是良性的肿瘤呢？比如纤维瘤，通常不会有疼痛，但这个患者有压痛，也不太对。

再想想，如果包块是恶性肿瘤，比如转移性肝癌，可能会有疼痛，但这个患者已经五天了，腹痛减轻了，可能不太像。另外，如果是急性阑尾炎，通常不会有包块，因为阑尾炎通常会伴随急性炎症，比如发热，而且包块不太常见。

那是不是有其他的可能性呢？比如包块是炎症性，比如急性炎症导致的肿胀，比如急性肾炎或者急性尿路感染。不过，患者已经有五天了，腹痛减轻了，而且发热，这可能不太符合急性炎症的表现。

那么，剩下的可能性可能是什么呢？或许包块是恶性肿瘤，比如肝细胞癌。这种情况下，患者可能会有腹痛和发热，但需要考虑是否有转移性症状。

在考虑这些可能性之后，我觉得包块可能与恶性肿瘤有关，特别是转移性肝癌。这种情况下，患者可能会有腹痛和发热，而包块可能在腹部的右侧。

所以，我应该建议进一步的检查，比如B超或者穿刺检查，来确认包块的性质，以及是否需要进一步的治疗。

总的来说，患者已经有五天的急性阑尾炎，腹痛减轻，发热，右下腹有压痛的包块，可能需要考虑包块是否是恶性肿瘤，特别是肝癌。建议进一步检查以确认包块的性质，并考虑是否需要进一步治疗。
</think>
根据患者的病情描述，他已发病5天，腹痛稍有减轻，但发热依旧，并在体检时发现右下腹有压痛的包块。这种情况下，包块可能是恶性肿瘤，尤其是转移性肝癌。这种情况下，患者通常会有腹痛和发热等症状。

为了进一步确认包块的性质，建议进行B超检查，以观察包块的形态和大小，以及是否有其他异常特征。同时，穿刺检查也是一个重要的步骤，可以确定包块的组织性质。此外，如果包块的大小较大，或伴随有其他症状，如发热、肿胀等，可能需要进一步的治疗和管理。

综上所述，建议对包块进行进一步的检查和评估，以确认其性质，并根据检查结果采取相应的治疗措施。如果包块与恶性肿瘤相关，可能需要进行手术切除或其他针对性治疗。



In [19]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [20]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
哦，这位60岁的男性患者，右胸疼，而且X线检查显示右肋膈角消失，这让我想到肺结核。对了，肺结核通常会带来胸水，所以我想他可能有胸水。

现在，我需要弄清楚这个胸水的性质到底是什么。要了解胸水的性质，通常需要做一些实验室检查。让我想想，这些检查能提供什么信息。

首先，血常规检查能告诉我血液里的细胞数量和类型。比如，如果白细胞增多，可能说明有炎症，而中性粒细胞增多可能和结核有关。不过，这可能不太直接能告诉我胸水的内容。

接下来，尿常规检查能告诉我尿液中的成分。如果发现有蛋白，可能说明有感染，或者有结核的迹象。不过，这也可能不太直接。

哦，对了，胸部穿刺检查，这可是个好办法。胸部穿刺可以收集到胸水样本，然后进行进一步的检测。这能直接告诉我胸水的成分，比如里面有没有细菌、病毒或者蛋白质，这样就能更清楚地了解胸水的性质。

哦，对了，胸水的培养和细菌学检查也是很重要的。如果胸水培养出来的是结核杆菌，那就说明是结核性胸水。这能更明确地确认诊断。

最后，血清学检查，比如抗核抗体，可以告诉我结核的情况。如果抗核抗体增高，那基本上可以确认结核的存在。

哦，对了，影像学检查比如超声或CT，也能提供更直观的信息，比如胸水的分布和数量。

嗯，综合来看，胸部穿刺和细菌学检查应该是最直接和最有效的方法。它们能让我更清楚地了解胸水的性质，从而做出正确的诊断和治疗。
</think>
对于一名60岁男性患者，右胸疼痛并伴有右侧胸腔积液的X线检查结果，提示可能存在结核性胸水。为了了解胸水的性质，以下实验室检查能提供更有帮助的信息：

1. **胸部穿刺**：通过胸腔穿刺可以获取胸水样本，进一步分析其成分。这有助于明确胸水中的细菌、病毒或蛋白质含量。

2. **胸水培养和细菌学检查**：培养胸水样本并进行细菌学检查，可以确定胸水是否含有结核杆菌或其他病原体，从而确认结核的存在。

3. **抗核抗体检测**：通过抗核抗体检测，可以评估结核抗原的水平，进一步确认结核感染的存在。

4. **影像学检查**：超声或CT影像可以提供胸水分布和量的直观信息，帮助评估胸水的性质。

综上所述，胸部穿刺和细菌学检查是了解胸水性质的最直接和有效的方法。这些检查能帮助确认结核性胸水的诊断，并指导

In [21]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这个患者28岁，是个程序员，常年熬夜，最近感觉头晕目眩，还有一点恶心。嗯，首先想到的是他可能有高血压，因为长期熬夜可能导致血压升高。但是，如果他的症状只是头晕和恶心，这可能还不够，因为高血压通常会有更多的症状，比如心跳快、头痛、眼花，等等。

再想想，他最近的活动可能和压力大有关。压力大可能导致血压上升，所以长期的压力和熬夜可能导致高血压，这似乎是合理的。不过，他的症状还不是很明显，所以可能需要更多的症状来支持高血压的诊断。

嗯，还有一种可能性是贫血，因为他如果经常熬夜，可能导致铁摄入不足，从而引起贫血。贫血通常会让人头晕，而且容易感到疲劳。不过，如果他的贫血是由于长期压力和熬夜导致的，那么他可能需要补充铁剂。

嗯，再想想，还有一种可能性是心脑血管疾病，比如脑供血不足。如果他的症状是头晕和恶心，这可能与脑供血不足有关。这种情况通常会伴随头晕、恶心和失眠等症状，尤其是在长时间工作后。

不过，我得再仔细想想，这些症状是否真的能支持这些诊断。如果他的症状只是头晕和恶心，没有其他明显症状，那么高血压和贫血的可能性似乎更高，因为它们通常会伴随这些症状。而心脑血管疾病，比如脑供血不足，通常会有更严重的症状，比如头痛、失眠，而这些症状可能更符合他的情况。

嗯，综合考虑，我觉得他可能有高血压，因为他的症状符合高血压的典型症状，尤其是头晕和恶心。不过，也有可能是贫血，尤其是长期压力和熬夜可能导致铁摄入不足。因此，我需要更多的信息来准确诊断。

嗯，看来我需要进一步的检查，比如血压、心率、血糖和血常规检查，以确认这些可能性。因为这些检查可以提供更多的信息，帮助明确诊断。不过，现在我只能根据症状来推测，所以可能需要更多的线索来确认。

嗯，总之，基于他的症状，高血压和贫血都是可能的诊断，但可能需要进一步的检查来明确。
</think>
根据该患者的症状和活动情况，最可能的诊断是高血压和贫血。高血压通常会导致头晕、恶心等症状，而贫血也常常与长期压力和熬夜导致的铁摄入不足有关。为了准确诊断，建议进行血压、心率、血糖和血常规检查，以确认这些可能性。这些检查可以提供更多的信息，帮助明确诊断。
